In [1]:
#!pip install geopyspark
#https://pypi.org/project/geopyspark/
#https://geopyspark.readthedocs.io/en/latest/guides/visualization.html

In [2]:
#!export SPARK_HOME="/usr/local/Cellar/apache-spark/2.4.4/libexec/"
!export SPARK_HOME="/opt/conda/lib/python3.7/site-packages/pyspark/"

In [3]:
# Importing our libraries
import os
os.environ["SPARK_HOME"] = "/opt/conda/lib/python3.7/site-packages/pyspark/"
#"/usr/local/Cellar/apache-spark/2.4.4/libexec/"
import geopyspark as gps
import numpy as np
import pyproj
import fiona
import folium

from functools import partial
from shapely.geometry import Point
from shapely.ops import transform
from pyspark import SparkContext, StorageLevel
from pyspark.sql import SparkSession

In [4]:
#!geopyspark install-jar
#
#jarspath = "/usr/local/Cellar/apache-spark/2.4.4/libexec/jars/"
#!geopyspark install-jar -p ""/opt/conda/lib/python3.7/site-packages/pyspark/jars/"
#cp geopyspark/jars/geotrellis-backend-assembly-0.4.3.jar pyspark/jars

In [5]:
#!wget https://s3.amazonaws.com/geopyspark-test/example-files/cropped.tif

In [6]:
#!pip install colortools

In [7]:
import geopyspark as gps
import matplotlib.pyplot as plt

from colortools import Color
from pyspark import SparkContext

%matplotlib inline

conf = gps.geopyspark_conf(master="local[*]", appName="visualization")

pysc = SparkContext(conf=conf)


In [8]:
#raster_layer = gps.geotiff.get(layer_type=gps.LayerType.SPATIAL, uri="file:///tmp/cropped.tif")

#raster_layer = gps.geotiff.get(layer_type=gps.LayerType.SPATIAL, uri="tmp/cropped.tif")
#tiled_layer = raster_layer.tile_to_layout(layout=gps.GlobalLayout(), target_crs=3857)

In [9]:
import datetime
import geopyspark as gps
import numpy as np

#from pyspark import SparkContext
from shapely.geometry import MultiPolygon, box

#conf = gps.geopyspark_conf(master="local[*]", appName="layers")
#pysc = SparkContext(conf=conf)

In [10]:
# Setting up the Spatial Data to be used in this example

pathtif = "/home/jovyan/work/bigdata/geospatial_libs/tmp/cropped.tif"


spatial_raster_layer = gps.geotiff.get(layer_type=gps.LayerType.SPATIAL, uri="./temp/cropped.tif")
#spatial_raster_layer = gps.geotiff.get(layer_type=gps.LayerType.SPATIAL, uri=pathtif)
#spatial_raster_layer = gps.geotiff.get(layer_type=gps.LayerType.SPATIAL, uri="file:////home/jovyan/work/bigdata/geospatial_libs/temp/cropped.tif")

spatial_tiled_layer = spatial_raster_layer.tile_to_layout(layout=gps.GlobalLayout(), target_crs=3857)

# Setting up the Spatial-Temporal Data to be used in this example

def make_raster(x, y, v, cols=4, rows=4, crs=4326):
    cells = np.zeros((1, rows, cols), dtype='float32')
    cells.fill(v)
    # extent of a single cell is 1
    extent = gps.TemporalProjectedExtent(extent = gps.Extent(x, y, x + cols, y + rows),
                                         epsg=crs,
                                         instant=datetime.datetime.now())

    return (extent, gps.Tile.from_numpy_array(cells))

layer = [
    make_raster(0, 0, v=1),
    make_raster(3, 2, v=2),
    make_raster(6, 0, v=3)
]

spatial_tiled_layer

#rdd = pysc.parallelize(layer)
#space_time_raster_layer = gps.RasterLayer.from_numpy_rdd(gps.LayerType.SPACETIME, rdd)
#space_time_tiled_layer = space_time_raster_layer.tile_to_layout(layout=gps.GlobalLayout(tile_size=5))
#space_time_pyramid = space_time_tiled_layer.pyramid()

TiledRasterLayer(layer_type=LayerType.SPATIAL, zoom_level=11, is_floating_point_layer=False)

In [11]:
pyramided = spatial_tiled_layer.pyramid()

In [12]:
gps.Pyramid([spatial_tiled_layer.tile_to_layout(gps.GlobalLayout(zoom=x)) for x in range(0, 13)])

Pyramid(layer_type=LayerType.SPATIAL, max_zoom=12, num_levels=13, is_cached=False)

In [13]:
gps.Pyramid({x: spatial_tiled_layer.tile_to_layout(gps.GlobalLayout(zoom=x)) for x in range(0, 13)})

Pyramid(layer_type=LayerType.SPATIAL, max_zoom=12, num_levels=13, is_cached=False)

In [14]:

%matplotlib inline

hist = pyramided.get_histogram()
hist